In [18]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from statsmodels.nonparametric.kernel_density import KDEMultivariate, KDEMultivariateConditional
import pandas as pd

In [19]:
df = sns.load_dataset("iris")

dic = {'setosa': 0, 'versicolor': 1, 'virginica': 2}
df['species'] = df['species'].map(dic)

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [49]:
X = df.iloc[:, :-1].to_numpy()
y = df.iloc[:, -1].to_numpy()

In [77]:
def Bayes_classifier(X, y, dep_type, indep_type):

    kde_Xc = KDEMultivariateConditional(endog=X, exog=y,
                                        dep_type=dep_type, indep_type=indep_type,
                                        bw='normal_reference').pdf
    kde_X = KDEMultivariate(X, var_type=dep_type).pdf
    Prior = np.unique(y, return_counts=True)[1] / y.size

    PXc = np.array([kde_Xc(X[0, :], [i]) for i in range(3)])
    PcX = PXc * Prior / kde_X(X[0, :])

    return PcX

In [79]:
Bayes_classifier(X, y, 'cccc', 'o')

array([0.59129795, 0.13441514, 0.07273842])

In [64]:
dep_type, indep_type = 'cccc', 'o'

kde_Xc = KDEMultivariateConditional(endog=X, exog=y,
                                        dep_type=dep_type, indep_type=indep_type,
                                        bw='normal_reference').pdf
kde_X = KDEMultivariate(X, var_type=dep_type).pdf
Prior = np.unique(y, return_counts=True)[1] / y.size

PXc = np.array([kde_Xc(X[:, :], [i] * y.size) for i in range(3)])
PcX = PXc * Prior / kde_X(X[:, :])

ValueError: operands could not be broadcast together with shapes (3,150) (3,) 

In [75]:
(np.argmax(((PXc.T * Prior).T / kde_X(X[:, :])).T, axis=1) == y).sum() / y.size

0.9666666666666667

In [63]:
kde_Xc(X[:, :], [0]*150)

array([0.13456223, 0.093696  , 0.1221964 , 0.10551239, 0.1230745 ,
       0.06700751, 0.11204455, 0.14070454, 0.05994416, 0.11064045,
       0.09693333, 0.13278892, 0.0927639 , 0.06169041, 0.03699528,
       0.01518958, 0.06682341, 0.13492261, 0.05961312, 0.09079043,
       0.10653979, 0.10544928, 0.08039461, 0.11305098, 0.12357617,
       0.08868713, 0.13356876, 0.13023853, 0.13096682, 0.12232798,
       0.11366226, 0.1044521 , 0.03960118, 0.03135707, 0.11416638,
       0.12334627, 0.09616081, 0.1128496 , 0.07472396, 0.13799148,
       0.13402843, 0.00727011, 0.09089284, 0.10438929, 0.0819805 ,
       0.09351155, 0.08958856, 0.11501899, 0.10377154, 0.13747304,
       0.01271778, 0.021345  , 0.01804571, 0.0142763 , 0.02627003,
       0.029785  , 0.01693234, 0.00919377, 0.02371353, 0.01711462,
       0.00426303, 0.02854008, 0.00889356, 0.03234862, 0.02247946,
       0.02024102, 0.02389695, 0.02410117, 0.00839559, 0.02131564,
       0.01742014, 0.02907513, 0.01720481, 0.02863352, 0.02707

In [93]:
def Naive_Bayes_classifier(X, y, dep_type, indep_type):
    n_classes = len(np.unique(y))
    n_features = X.shape[1]

    kde_Xc = []
    kde_X = []

    for i in range(n_features):
        kde_Xc.append(KDEMultivariateConditional(endog=X[:, i], exog=y, 
                                                 dep_type=dep_type, 
                                                 indep_type=indep_type, 
                                                 bw='normal_reference').pdf)
        kde_X.append(KDEMultivariate(X[:, i], var_type=dep_type).pdf)

    Prior = np.unique(y, return_counts=True)[1] / y.size

    c = np.zeros(n_classes)
    for i in range(n_classes):
        tmp = 1
        for j in range(n_features):
            tmp *= kde_Xc[j]([X[0, j]], [i]) / kde_X[j]([X[0, j]])
        c[i] = tmp * Prior[i]

    return c

In [95]:
def Naive_Bayes_classifier(X, y, dep_type, indep_type):
    n_classes = len(np.unique(y))
    n_features = X.shape[1]

    kde_Xc = []

    for i in range(n_features):
        kde_Xc.append(KDEMultivariateConditional(endog=X[:, i], exog=y, 
                                                 dep_type=dep_type, 
                                                 indep_type=indep_type, 
                                                 bw='normal_reference').pdf)
        
    kde_X = KDEMultivariate(X, var_type='cccc').pdf

    Prior = np.unique(y, return_counts=True)[1] / y.size

    c = np.zeros(n_classes)
    for i in range(n_classes):
        tmp = 1
        for j in range(n_features):
            tmp *= kde_Xc[j]([X[0, j]], [i])
        c[i] = tmp * Prior[i] / kde_X(X[0, :])

    return c

In [98]:
Naive_Bayes_classifier(X, y, 'c', 'o')

array([7.43143307e-01, 4.09563514e-03, 3.12770498e-04])